<span type="title">单件模式和命令模式</span> | <span type="update">2018-09-02</span> - Version <span type="version">1.1</span>
    
    
<span type="intro"><p class="card-text">本章介绍单件模式和命令模式。对于单件模式而言，其强调一个类只能有一个实例，对此，我们使用私有化构造器并且提供一个实例变量，通过一个静态方法获取这个变量的方式实现。单件面临多线程问题，解决方案是使用全局变量或者使用锁，或者使用双重检查加锁来实现快速，安全的多线程单件模式。单件模式属于“提供服务”中“使用组合”的设计类别，和其属于同一子类别的有策略模式，状态模式。同样具有一个实例化对象的有代理模式和装饰器模式，不过后者更改了接口，达到了不同的目的。</p><p class="card-text">命令模式通过将请求封装成对象，然后单个或者组合的参数化其余对象以提供服务，命令模式强调动作的发出者和动作的执行者的解耦。这种设计模式支持日志、队列、撤销等操作。</p></span>

# 单件模式

在一些情况下，我们需要一个类只保留一个实例，比如线程池、注册表等资源敏感型组件中。这种需求可以使用单件设计模式。单件的可能替代有：全局变量、静态变量，但是这些方式都或多或少有一些问题，尤其是在工程和多线程方便。

## 单线程版本

一个典型的单线程单件模式要考虑到使用代码人员的愚蠢问题：

```java
public class Singleton {
    private static Singleton instance;
    private Singleton() {}
    public static Singleton getInstance() {
        if (instance == null) {
            instance = new Singleton();
        } return instance;
    }
}
```

这种方式是典型的单件模式，使用 private 避免了公开构造，getInstance静态方法可以直接返回一个实例，并且当多次调用时，始终返回一个实例。这种设计的缺点在于 private 不能继承，如果改为 protected，那么又违反了单件的初衷。

你可能想说 instance 变量直接 `instance = new Singleton()` 更方便，不用写条件语句判断，但是这样的话，构造对象的压力很大（也就是Singleton.class 加载时的压力很大）。

## 多线程版本

除此之外，这个经典模式的问题时：在多线程情况下可能产生多个instance。解决办法之一是：`public static synchronized Singleton getInstance() {}` 改写成这种同步的方法即可解决多线程问题。但是，这对性能有影响（100倍左右），如果有性能需求，则可以考虑使用急切模式，也就是直接使用全局变量先初始化 static 的 instance 域，这样JVM加载Singleton类的压力会变大。当然，如果这不是瓶颈的话，其实无所谓，使用同步，或者全局变量均可解决多线程问题。

但是，最好的解决办法是，使用双重检查加锁：

```java
public class Singleton {
    private volatile static Singleton instance;
    private Singleton() {}
    public static Singleton getInstance() {
        if (instance == null) {
            synchronized (Singleton.class) {
                if (instance == null) {
                    instance = new Singleton();
                }
            }
        } return instance;
    }
}
```
这种方法仅在首次调用 Singletion 类时进入 synchronized 模式，创建实例，此外，均直接返回实例，避免了频繁调用 getInstance 的性能问题。注意，这里的 instance 全局变量使用了 `volatile` 关键字，这个关键字能够确保多线程中互相得知这个变量的变化情况。synchronized 则是确保在同一时间只有一个线程进入其内部代码块。

# 命令模式

命令模式指的是：**将请求封装成对象**。这允许我们使用不同的请求、队列、日志请求以参数化其他对象。命令模式支持撤销操作，可以实现日志和事务系统。

命令模式提供了一种发出请求和执行请求的对象之间的解耦。其中，请求被发出到命令对象，命令对象封装了一个或者一组接收者的动作，通过execute接口执行接收者。在这种模式中，动作的发出者和动作的执行者之间被命令对象隔开。就好像老板命令秘书干活，只给一个简短的指示，秘书负责协调其余人（对象），航班以及其余活动等等。

下面展示了使用命令模式实现自动化家具，这是一个遥控器，可以控制很多家电。

首先是各个厂商提供的接口：

```java
class Conditioner {
    int temp;
    void setTemp(int temp) {
        this.temp = temp;
    }
    void shutdown() {
        //do something shutdown
    }
}
class Light {
    void setOn() {};
    void setOff() {};
}
class TV {
    void powerOn() {};
    void powerOff() {};
    void changeTo(int channel) {
        // do something to change channel
    };
}
```
接着是命令接口和一些控制电器的命令：

```java
interface Command {
    void execute();
    void undo();
}
class ConditionerOnCommand implements Command {
    Conditioner machine;
    int lastTemp;
    ConditionerOnCommand(Conditioner machine) {
        this.machine = machine;
        lastTemp = 27;
    }
    void execute() {
        this.setTemp(lastTemp);
    }
    void undo() {
        machine.shutdown();
    }
}
class ConditionerOffCommand implements Command {
    Conditioner machine;
    int lastTemp;
    ConditionerOnCommand(Conditioner machine) {
        this.machine = machine;
        lastTemp = 27;
    }
    void execute() {
        machine.shutdown();
    }
    void undo() {
        this.setTemp(lastTemp);
    }
}
class LightOnCommand implements Command {
    Light light;
    LightOnCommand(Light light) { this.light = light; }
    void execute() { light.on(); }
    void undo() { light.off(); }
}
class LightOffCommand implements Command {
    Light light;
    LightOnCommand(Light light) { this.light = light; }
    void execute() { light.off(); }
    void undo() { light.on(); }
}
class TVOnCommand implements Commmand {
    Tv tv;
    TVOnCommand(Tv tv) { this.tv = tv; }
    void execute() { tv.on(); }
    void undo() { tv.off(); }
}
class TVOffCommand implements Commmand {
    Tv tv;
    TVOnCommand(Tv tv) { this.tv = tv; }
    void execute() { tv.off(); }
    void undo() { tv.on(); }
}
class TVChangeCommand implements Command {
    Tv tv;
    int lastChannel;
    TVChangeCommand(Tv tv) { this.tv = tv; }
    void execute() {
        lastChannel++;
        checkMaxChannel();
        tv.changeTo(lastChannel)
    }
    void undo() {
        tv.changeTo(--lastChannel)
    }
}
```

最后是遥控器，通过命令控制家电：

```java
class Remote {
    ConditionerOnCommand coc;
    ConditionerOffCommand cfc;
    TVOnCommand toc;
    TVOffCommand tfc;
    TVChangeCommand tcc;
    LightOnCommand loc;
    LightOffCommand lfc;
    public Remote() {
        Conditioner c = new Conditioner();
        Tv tv = new Tv();
        Light light = new Light();
        coc = new ConditionerOnCommand(c);
        cfc = new ConditionerOffCommand(c);
        toc = new TVOnCommand(tv);
        tfc = new TVOffCommand(tv);
        tcc = new TVChangeCommand(tv);
        loc = new LightOnCommand(light);
        lfc = new LightOffCommand(light);
    }
    void partyOn() {
        coc.execute();
        toc.execute();
        loc.execute();
    }
    void leaveHome() {
        cfc.execute();
        tfc.execute();
        lfc.execute();
    }
    void button1() { coc.execute(); }
    void button2() { cfc.execute(); }
    void button3() { toc.execute(); }
    void button4() { tfc.execute(); }
    void button5() { loc.execute(); }
    void button6() { lfc.execute(); }
    void changeChannel() { tcc.execute(); }
}
```